Merge JSONL 

In [4]:
import json

files = [
    r"D:/chatbot_covid19_intern_procodecg/data/processed/covid_19_to_narative.jsonl",
    r"D:/chatbot_covid19_intern_procodecg/data/processed/knowladge_covid19_indonesia.jsonl"
]

merge_file = r"D:/chatbot_covid19_intern_procodecg/data/processed/dataset_covid__merge.jsonl"

with open(merge_file, "w", encoding="utf-8") as outfile:
    for path in files :
        with open(path, "r", encoding="utf-8") as infile:
            for line in infile:
                outfile.write(line)
                
print(f"semua chunk telah digabung di: {merge_file}")



semua chunk telah digabung di: D:/chatbot_covid19_intern_procodecg/data/processed/dataset_covid__merge.jsonl


In [1]:
from sentence_transformers import SentenceTransformer
import json
import numpy as np

chunks_path = r"D:/chatbot_covid19_intern_procodecg/data/processed/knowladge_covid19_indonesia.jsonl"
embedding_path = r"D:/chatbot_covid19_intern_procodecg/vector_db/faiss__bahasa_metadata.json"

# 1️⃣ Load model lokal untuk embedding
print("🧠 Memuat model SentenceTransformer ...")
model = SentenceTransformer("indobenchmark/indobert-base-p1")

embeddings = []

# 2️⃣ Baca dan buat embedding per chunk
with open(chunks_path, "r", encoding="utf-8") as f:
    for i, line in enumerate(f):
        data = json.loads(line)
        text = data["text"]

        emb = model.encode(text).tolist()
        data["embedding"] = emb

        embeddings.append(data)
        print(f"✅ Chunk {i+1} selesai di-embed")

# 3️⃣ Simpan hasilnya ke JSONL
with open(embedding_path, "w", encoding="utf-8") as f:
    for item in embeddings:
        f.write(json.dumps(item, ensure_ascii=False) + "\n")

print(f"💾 Semua embedding tersimpan di: {embedding_path}")


🧠 Memuat model SentenceTransformer ...


No sentence-transformers model found with name indobenchmark/indobert-base-p1. Creating a new one with MEAN pooling.


config.json: 0.00B [00:00, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


pytorch_model.bin:   0%|          | 0.00/498M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

d:\chatbot_covid19_intern_procodecg\.venvr\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


✅ Chunk 1 selesai di-embed
✅ Chunk 2 selesai di-embed
✅ Chunk 3 selesai di-embed
✅ Chunk 4 selesai di-embed
✅ Chunk 5 selesai di-embed
✅ Chunk 6 selesai di-embed
✅ Chunk 7 selesai di-embed
✅ Chunk 8 selesai di-embed
✅ Chunk 9 selesai di-embed
✅ Chunk 10 selesai di-embed
✅ Chunk 11 selesai di-embed
✅ Chunk 12 selesai di-embed
✅ Chunk 13 selesai di-embed
✅ Chunk 14 selesai di-embed
✅ Chunk 15 selesai di-embed
✅ Chunk 16 selesai di-embed
✅ Chunk 17 selesai di-embed
✅ Chunk 18 selesai di-embed
✅ Chunk 19 selesai di-embed
✅ Chunk 20 selesai di-embed
💾 Semua embedding tersimpan di: D:/chatbot_covid19_intern_procodecg/vector_db/faiss__bahasa_metadata.json


In [ ]:
from sentence_transformers import SentenceTransformer
import json
import numpy as np
import faiss
import os
from tqdm import tqdm

class COVID19EmbeddingGenerator:
    def __init__(self, model_name="thenlper/gte-base"):
        """
        Initialize embedding generator dengan model pilihan
        
        Model options:
        - "thenlper/gte-base" (Recommended)
        - "all-mpnet-base-v2"
        - "intfloat/multilingual-e5-base" 
        - "sentence-transformers/paraphrase-multilingual-mpnet-base-v2"
        - "all-MiniLM-L12-v2" (Lightweight)
        """
        print(f"🧠 Loading model: {model_name}")
        self.model = SentenceTransformer(model_name)
        self.model_name = model_name
        
    def load_chunks(self, chunks_path):
        """Load chunks dari JSONL file"""
        chunks = []
        with open(chunks_path, "r", encoding="utf-8") as f:
            for line in f:
                data = json.loads(line.strip())
                chunks.append(data)
        print(f"📖 Loaded {len(chunks)} chunks from {chunks_path}")
        return chunks
    
    def create_embeddings(self, chunks_path, output_path, batch_size=16):
        """Buat embeddings untuk semua chunks"""
        chunks = self.load_chunks(chunks_path)
        
        print("🔍 Creating embeddings...")
        all_embeddings = []
        
        for i in tqdm(range(0, len(chunks), batch_size), desc="Processing batches"):
            batch_chunks = chunks[i:i + batch_size]
            batch_texts = [chunk["text"] for chunk in batch_chunks]
            
            # Buat embeddings dengan normalisasi
            batch_embeddings = self.model.encode(
                batch_texts,
                normalize_embeddings=True,  # ⭐ PENTING untuk reduce halusinasi
                show_progress_bar=False,
                convert_to_tensor=False
            )
            
            # Tambah metadata
            for j, chunk in enumerate(batch_chunks):
                chunk["embedding"] = batch_embeddings[j].tolist()
                chunk["embedding_model"] = self.model_name
                chunk["chunk_id"] = f"chunk_{i+j}"
                chunk["embedding_version"] = "v2_optimized"
            
            all_embeddings.extend(batch_chunks)
        
        # Simpan hasil
        self._save_embeddings(all_embeddings, output_path)
        return all_embeddings
    
    def _save_embeddings(self, embeddings, output_path):
        """Save embeddings ke file JSONL"""
        os.makedirs(os.path.dirname(output_path), exist_ok=True)
        
        with open(output_path, "w", encoding="utf-8") as f:
            for item in embeddings:
                f.write(json.dumps(item, ensure_ascii=False) + "\n")
        print(f"💾 Embeddings saved to: {output_path}")
        print(f"📊 Total embeddings: {len(embeddings)}")

def main():
    # 🔧 CONFIGURATION - GANTI DI SINI UNTUK COBA MODEL BERBEDA
    MODEL_CONFIGS = {
        "gte-base": "thenlper/gte-base",  # Recommended
        "mpnet-base": "all-mpnet-base-v2",
        "multilingual-e5": "intfloat/multilingual-e5-base", 
        "multilingual-mpnet": "sentence-transformers/paraphrase-multilingual-mpnet-base-v2",
        "minilm": "all-MiniLM-L12-v2"  # Lightweight
    }
    
    # Pilih model yang mau dicoba
    SELECTED_MODEL = "gte-base"  # 🔄 GANTI INI UNTUK COBA MODEL LAIN
    
    # Path files
    CHUNKS_PATH = r"D:/chatbot_covid19_intern_procodecg/data/processed/knowladge_covid19_indonesia.jsonl"
    OUTPUT_PATH = r"D:/chatbot_covid19_intern_procodecg/vector_db/embeddings_optimized.jsonl"
    
    print(f"🎯 Using model: {SELECTED_MODEL} -> {MODEL_CONFIGS[SELECTED_MODEL]}")
    
    # Initialize generator
    generator = COVID19EmbeddingGenerator(MODEL_CONFIGS[SELECTED_MODEL])
    
    # Create embeddings
    embeddings = generator.create_embeddings(
        chunks_path=CHUNKS_PATH,
        output_path=OUTPUT_PATH,
        batch_size=8  # Adjust berdasarkan GPU memory
    )
    
    print("✅ Embedding generation completed!")

if __name__ == "__main__":
    main()

In [3]:
# create_embeddings.py
import os
import json
import numpy as np
import faiss
from sentence_transformers import SentenceTransformer
import re

def setup_paths():
    """Setup project paths - FIXED untuk Jupyter/Colab"""
    # Untuk Jupyter/Colab, tentukan path manual
    project_root = r"D:/chatbot_covid19_intern_procodecg"
    faiss_dir = os.path.join(project_root, "faiss")
    data_dir = os.path.join(project_root, "data")
    
    os.makedirs(faiss_dir, exist_ok=True)
    os.makedirs(data_dir, exist_ok=True)
    
    return {
        'project_root': project_root,
        'faiss_dir': faiss_dir,
        'data_dir': data_dir,
        'index_path': os.path.join(faiss_dir, "faiss_textcovid19.index"),
        'texts_path': os.path.join(faiss_dir, "faiss_textcovid19_texts.json"),
        'chunks_path': os.path.join(data_dir, "chunks.json"),
        'jsonl_path': r"D:/chatbot_covid19_intern_procodecg/data/processed/knowladge_covid19_indonesia.jsonl"
    }

def load_chunks_from_jsonl(jsonl_path):
    """Load chunks dari file JSONL"""
    print(f"📖 Loading chunks from JSONL: {jsonl_path}")
    
    if not os.path.exists(jsonl_path):
        print(f"❌ JSONL file not found: {jsonl_path}")
        return []
    
    chunks = []
    try:
        with open(jsonl_path, 'r', encoding='utf-8') as f:
            for i, line in enumerate(f):
                line = line.strip()
                if line:
                    try:
                        data = json.loads(line)
                        # Extract text dari berbagai kemungkinan field names
                        text = ""
                        if 'text' in data:
                            text = data['text']
                        elif 'content' in data:
                            text = data['content']
                        elif 'chunk' in data:
                            text = data['chunk']
                        elif 'page_content' in data:
                            text = data['page_content']
                        else:
                            # Jika tidak ada field yang dikenal, gunakan seluruh data sebagai string
                            text = str(data)
                        
                        if text and len(text.strip()) > 10:  # Minimal 10 karakter
                            chunks.append({
                                'id': len(chunks),
                                'text': text.strip(),
                                'source': data.get('source', 'unknown'),
                                'metadata': data.get('metadata', {})
                            })
                            
                    except json.JSONDecodeError as e:
                        print(f"⚠️ JSON decode error on line {i}: {e}")
                        continue
        
        print(f"✅ Loaded {len(chunks)} chunks from JSONL")
        
        # Show sample chunks
        print(f"\n📝 Sample chunks from JSONL:")
        for i in range(min(5, len(chunks))):
            print(f"   {i+1}. {chunks[i]['text'][:100]}...")
            
        return chunks
        
    except Exception as e:
        print(f"❌ Error loading JSONL: {e}")
        import traceback
        traceback.print_exc()
        return []

def create_embeddings(chunks):
    """Create embeddings untuk semua chunks"""
    print("🔄 Loading embedding model...")
    
    # Gunakan model yang sama dengan retriever
    model = SentenceTransformer('paraphrase-multilingual-mpnet-base-v2')
    
    print("🔄 Creating embeddings...")
    texts = [chunk['text'] for chunk in chunks]
    
    # Create embeddings
    embeddings = model.encode(texts, convert_to_numpy=True, show_progress_bar=True)
    embeddings = embeddings.astype('float32')
    
    print(f"✅ Created embeddings: {embeddings.shape}")
    return embeddings

def create_faiss_index(embeddings):
    """Create FAISS index dari embeddings"""
    print("🔄 Creating FAISS index...")
    
    # Dimensionality
    dimension = embeddings.shape[1]
    
    # Create FAISS index
    index = faiss.IndexFlatIP(dimension)  # Inner Product untuk cosine similarity
    
    # Normalize vectors untuk cosine similarity
    faiss.normalize_L2(embeddings)
    
    # Add ke index
    index.add(embeddings)
    
    print(f"✅ FAISS index created: {index.ntotal} vectors")
    return index

def save_data(chunks, index, paths):
    """Save semua data ke file"""
    print("🔄 Saving data...")
    
    # Save chunks dengan metadata lengkap
    with open(paths['chunks_path'], 'w', encoding='utf-8') as f:
        json.dump(chunks, f, ensure_ascii=False, indent=2)
    
    # Save texts untuk retriever (hanya teks saja)
    texts_for_retriever = [chunk['text'] for chunk in chunks]
    with open(paths['texts_path'], 'w', encoding='utf-8') as f:
        json.dump(texts_for_retriever, f, ensure_ascii=False, indent=2)
    
    # Save FAISS index
    faiss.write_index(index, paths['index_path'])
    
    print(f"✅ Saved {len(chunks)} chunks to: {paths['chunks_path']}")
    print(f"✅ Saved {len(texts_for_retriever)} texts to: {paths['texts_path']}")
    print(f"✅ Saved index to: {paths['index_path']}")

def verify_data(paths):
    """Verify bahwa data berhasil disimpan"""
    print("\n🔍 Verifying saved data...")
    
    # Check files exist
    for file_type, path in {
        'JSONL source': paths['jsonl_path'],
        'Chunks file': paths['chunks_path'],
        'Texts file': paths['texts_path'],
        'Index file': paths['index_path']
    }.items():
        exists = os.path.exists(path)
        print(f"   {file_type}: {'✅ EXISTS' if exists else '❌ MISSING'} - {path}")
    
    # Load and check chunks
    try:
        with open(paths['chunks_path'], 'r', encoding='utf-8') as f:
            chunks = json.load(f)
        print(f"   Chunks count: ✅ {len(chunks)}")
        
        # Show sample chunks
        print(f"\n📝 Sample saved chunks:")
        for i in range(min(3, len(chunks))):
            print(f"   {i+1}. {chunks[i]['text'][:80]}...")
            
    except Exception as e:
        print(f"   ❌ Error loading chunks: {e}")
    
    # Load and check index
    try:
        index = faiss.read_index(paths['index_path'])
        print(f"   FAISS vectors: ✅ {index.ntotal}")
    except Exception as e:
        print(f"   ❌ Error loading index: {e}")

def test_search_capability(paths):
    """Test kemampuan search setelah membuat embeddings"""
    print("\n🔍 Testing search capability...")
    
    try:
        # Load model
        model = SentenceTransformer('paraphrase-multilingual-mpnet-base-v2')
        
        # Load index dan texts
        index = faiss.read_index(paths['index_path'])
        with open(paths['texts_path'], 'r', encoding='utf-8') as f:
            texts = json.load(f)
        
        # Test queries
        test_queries = [
            "Siapa orang pertama yang divaksin di Indonesia?",
            "Apa vaksin kedua yang digunakan?",
            "Jenis vaksin COVID-19 di Indonesia",
            "Presiden Jokowi vaksin",
            "Gejala COVID-19",
            "Aplikasi PeduliLindungi"
        ]
        
        print(f"🧪 Testing {len(test_queries)} queries...")
        
        for query in test_queries:
            print(f"\n📝 Query: '{query}'")
            
            # Encode query
            query_embedding = model.encode([query], convert_to_numpy=True).astype("float32")
            faiss.normalize_L2(query_embedding)
            
            # Search
            scores, indices = index.search(query_embedding, 3)
            
            found_good_match = False
            for i, (idx, score) in enumerate(zip(indices[0], scores[0])):
                if 0 <= idx < len(texts) and score > 0.2:  # Threshold reasonable
                    text_preview = texts[idx][:120] + "..." if len(texts[idx]) > 120 else texts[idx]
                    print(f"   ✅ {i+1}. Score: {score:.3f} - {text_preview}")
                    found_good_match = True
            
            if not found_good_match:
                print("   ❌ NO GOOD MATCHES FOUND!")
                
    except Exception as e:
        print(f"❌ Error during search test: {e}")

def main():
    """Main function"""
    print("🚀 STARTING EMBEDDING CREATION FROM JSONL...")
    
    try:
        # Setup paths
        paths = setup_paths()
        print(f"📁 Project root: {paths['project_root']}")
        print(f"📁 FAISS dir: {paths['faiss_dir']}")
        print(f"📁 JSONL source: {paths['jsonl_path']}")
        
        # Step 1: Load chunks dari JSONL
        print("\n📖 Step 1: Loading chunks from JSONL...")
        chunks = load_chunks_from_jsonl(paths['jsonl_path'])
        
        if not chunks:
            print("❌ No chunks loaded from JSONL!")
            return
        
        # Step 2: Create embeddings
        print("\n🔤 Step 2: Creating embeddings...")
        embeddings = create_embeddings(chunks)
        
        # Step 3: Create FAISS index
        print("\n📊 Step 3: Creating FAISS index...")
        index = create_faiss_index(embeddings)
        
        # Step 4: Save data
        print("\n💾 Step 4: Saving data...")
        save_data(chunks, index, paths)
        
        # Step 5: Verify
        print("\n✅ Step 5: Verification...")
        verify_data(paths)
        
        # Step 6: Test search
        print("\n🔍 Step 6: Testing search capability...")
        test_search_capability(paths)
        
        print(f"\n🎉 EMBEDDING CREATION COMPLETED!")
        print(f"📊 Total chunks: {len(chunks)}")
        print(f"📊 Embedding dimension: {embeddings.shape[1]}")
        print(f"📊 FAISS index size: {index.ntotal} vectors")
        
        print(f"\n📋 NEXT STEPS:")
        print(f"1. Run your Streamlit app")
        print(f"2. Clear cache di halaman '🧹 Cache'")
        print(f"3. Test dengan: 'Siapa orang pertama yang divaksin?'")
        print(f"4. Test dengan: 'Apa vaksin kedua yang digunakan?'")
        
    except Exception as e:
        print(f"❌ ERROR: {e}")
        import traceback
        traceback.print_exc()

# Jalankan langsung jika di-run sebagai script
if __name__ == "__main__":
    main()

🚀 STARTING EMBEDDING CREATION FROM JSONL...
📁 Project root: D:/chatbot_covid19_intern_procodecg
📁 FAISS dir: D:/chatbot_covid19_intern_procodecg\faiss
📁 JSONL source: D:/chatbot_covid19_intern_procodecg/data/processed/knowladge_covid19_indonesia.jsonl

📖 Step 1: Loading chunks from JSONL...
📖 Loading chunks from JSONL: D:/chatbot_covid19_intern_procodecg/data/processed/knowladge_covid19_indonesia.jsonl
✅ Loaded 20 chunks from JSONL

📝 Sample chunks from JSONL:
   1. 1. Informasi Medis dan Kesehatan Publik
COVID-19 adalah penyakit menular yang disebabkan oleh virus ...
   2. 2. Program Vaksinasi Nasional
Pemerintah Indonesia memulai program vaksinasi nasional pada Januari 2...
   3. 3. Statistik dan Data Nasional
Selama pandemi, Indonesia secara rutin memperbarui data kasus COVID-1...
   4. 4. Panduan Aktivitas dan Mobilitas
Untuk menekan penyebaran virus, pemerintah memberlakukan berbagai...
   5. 5. Edukasi Publik dan Penanggulangan Hoaks
Pandemi COVID-19 juga diiringi oleh fenomena i

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

✅ Created embeddings: (20, 768)

📊 Step 3: Creating FAISS index...
🔄 Creating FAISS index...
✅ FAISS index created: 20 vectors

💾 Step 4: Saving data...
🔄 Saving data...
✅ Saved 20 chunks to: D:/chatbot_covid19_intern_procodecg\data\chunks.json
✅ Saved 20 texts to: D:/chatbot_covid19_intern_procodecg\faiss\faiss_textcovid19_texts.json
✅ Saved index to: D:/chatbot_covid19_intern_procodecg\faiss\faiss_textcovid19.index

✅ Step 5: Verification...

🔍 Verifying saved data...
   JSONL source: ✅ EXISTS - D:/chatbot_covid19_intern_procodecg/data/processed/knowladge_covid19_indonesia.jsonl
   Chunks file: ✅ EXISTS - D:/chatbot_covid19_intern_procodecg\data\chunks.json
   Texts file: ✅ EXISTS - D:/chatbot_covid19_intern_procodecg\faiss\faiss_textcovid19_texts.json
   Index file: ✅ EXISTS - D:/chatbot_covid19_intern_procodecg\faiss\faiss_textcovid19.index
   Chunks count: ✅ 20

📝 Sample saved chunks:
   1. 1. Informasi Medis dan Kesehatan Publik
COVID-19 adalah penyakit menular yang di...
   2. 2